In [9]:
# Importar librerías necesarias
import pandas as pd

# Ruta del dataset original
ruta_dataset = "Limpieza/data/df_unificado_limpio_actualizado.csv"

# Cargar el dataset original
df = pd.read_csv(ruta_dataset)

# Revisar estructura del dataset
print("Estructura del dataset original:")
print(df.info())

# Verificar valores únicos de columnas clave
print("Valores únicos de DEPARTAMENTO:", df["DEPARTAMENTO"].unique())
print("Valores únicos de MUNICIPIO:", df["MUNICIPIO"].unique())
print("Valores únicos de AÑO:", df["AÑO"].unique())

# Asegurarse de que las columnas clave no tengan valores nulos
df = df.dropna(subset=["DEPARTAMENTO", "MUNICIPIO", "AÑO", "No. ACCESOS FIJOS A INTERNET"])


Estructura del dataset original:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 917690 entries, 0 to 917689
Data columns (total 14 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   AÑO                           917690 non-null  int64  
 1   TRIMESTRE                     917690 non-null  int64  
 2   PROVEEDOR                     917690 non-null  object 
 3   CÓDIGO DANE                   917690 non-null  int64  
 4   DEPARTAMENTO                  917690 non-null  object 
 5   CÓDIGO DANE.1                 917690 non-null  int64  
 6   MUNICIPIO                     917690 non-null  object 
 7   SEGMENTO                      917690 non-null  object 
 8   TECNOLOGÍA                    917690 non-null  object 
 9   VELOCIDAD BAJADA              917690 non-null  float64
 10  VELOCIDAD SUBIDA              917690 non-null  float64
 11  No. ACCESOS FIJOS A INTERNET  917690 non-null  int64  
 12  Latitud    

In [10]:
# Definir la tasa de crecimiento anual y los años futuros
tasa_crecimiento = 0.05
anios_futuros = [2022, 2023, 2024, 2025]

# Crear una lista para almacenar los nuevos datos
nuevos_datos = []

# Generar datos futuros basados en la tasa de crecimiento
for (departamento, municipio), group in df.groupby(["DEPARTAMENTO", "MUNICIPIO"]):
    print(f"Procesando: Departamento={departamento}, Municipio={municipio}")

    # Verificar el grupo procesado
    print(f"Datos del grupo:\n{group[['AÑO', 'No. ACCESOS FIJOS A INTERNET']]}")

    # Filtrar los datos más recientes por municipio y departamento
    ultimo_anio = group["AÑO"].max()
    datos_ultimo_anio = group[group["AÑO"] == ultimo_anio]

    if datos_ultimo_anio.empty:
        print(f"Sin datos para {departamento}-{municipio}")
        continue

    # Obtener accesos actuales
    accesos_actuales = datos_ultimo_anio["No. ACCESOS FIJOS A INTERNET"].sum()
    print(f"Accesos actuales para el año {ultimo_anio}: {accesos_actuales}")

    for anio in anios_futuros:
        if anio <= ultimo_anio:
            # Ignorar años pasados o iguales al último año
            continue

        # Calcular accesos para el año futuro
        accesos_futuros = int(accesos_actuales * (1 + tasa_crecimiento))
        print(f"Generando para {anio}: {accesos_futuros} accesos.")

        nuevos_datos.append({
            "AÑO": anio,
            "DEPARTAMENTO": departamento,
            "MUNICIPIO": municipio,
            "No. ACCESOS FIJOS A INTERNET": accesos_futuros
        })

        # Actualizar accesos actuales
        accesos_actuales = accesos_futuros

# Crear un DataFrame con los nuevos datos generados
df_futuro = pd.DataFrame(nuevos_datos)


Procesando: Departamento=ANTIOQUIA, Municipio=AMALFI
Datos del grupo:
         AÑO  No. ACCESOS FIJOS A INTERNET
0       2021                             1
1       2021                             1
2       2021                             1
244     2021                             1
245     2021                             1
...      ...                           ...
903576  2024                             1
903577  2024                             1
903578  2024                             1
903579  2024                             6
903580  2024                             3

[1032 rows x 2 columns]
Accesos actuales para el año 2024: 1653
Generando para 2025: 1735 accesos.
Procesando: Departamento=ANTIOQUIA, Municipio=BELLO
Datos del grupo:
         AÑO  No. ACCESOS FIJOS A INTERNET
3       2021                             3
4       2021                            10
246     2021                             1
564     2021                             1
3325    2021                  

In [11]:
# Validar los datos futuros generados
print("Datos futuros generados (primeros 5 registros):")
print(df_futuro.head())
print(f"Total de registros generados: {len(df_futuro)}")

# Combinar los datos originales con los futuros
df_combinado = pd.concat([df, df_futuro], ignore_index=True)

# Guardar los datos combinados en un nuevo archivo CSV
nueva_ruta_dataset = "Limpieza/data/df_unificado_limpio_con_proyecciones.csv"
df_combinado.to_csv(nueva_ruta_dataset, index=False)

print(f"Datos combinados guardados en: {nueva_ruta_dataset}")
print(f"Total de filas después de agregar proyecciones: {len(df_combinado)}")


Datos futuros generados (primeros 5 registros):
    AÑO DEPARTAMENTO    MUNICIPIO  No. ACCESOS FIJOS A INTERNET
0  2025    ANTIOQUIA       AMALFI                          1735
1  2025    ANTIOQUIA        BELLO                        153326
2  2025    ANTIOQUIA       CAREPA                          6015
3  2025    ANTIOQUIA     CAUCASIA                         10291
4  2025    ANTIOQUIA  CAÑASGORDAS                          1141
Total de registros generados: 195
Datos combinados guardados en: Limpieza/data/df_unificado_limpio_con_proyecciones.csv
Total de filas después de agregar proyecciones: 917885


In [12]:
# Revisar datos por año en el dataset combinado
print("Resumen por Año:")
print(df_combinado.groupby("AÑO")["No. ACCESOS FIJOS A INTERNET"].sum())


Resumen por Año:
AÑO
2021     6420883
2022    13455755
2023    13767760
2024     3420224
2025     3591146
Name: No. ACCESOS FIJOS A INTERNET, dtype: int64
